# Day8_5: 다변량 시계열 분석 (VAR + 그랜저 인과성) - 정답

---

이 노트북은 Day8_5 실습 퀴즈의 정답과 상세 풀이를 제공합니다.

In [ ]:
# 필요한 라이브러리
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시계열 분석 라이브러리
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

# 경고 숨기기
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# 실습에 사용할 데이터 생성
np.random.seed(42)

# 날짜 인덱스 생성
dates = pd.date_range(start='2010-01-01', periods=150, freq='MS')

# Money: 추세 + 랜덤
money_trend = np.linspace(1000, 2500, 150)
money = money_trend + np.cumsum(np.random.randn(150) * 30)

# Spending: Money에 지연 반응 + 자체 트렌드
spending_trend = np.linspace(800, 1800, 150)
spending = spending_trend + 0.3 * np.roll(money, 3) + np.cumsum(np.random.randn(150) * 20)

# DataFrame 생성
df_econ = pd.DataFrame({
    'Money': money,
    'Spending': spending
}, index=dates)

# 차분 데이터
df_diff = df_econ.diff().dropna()

print("데이터 준비 완료!")
print(f"원본 데이터: {len(df_econ)}개")
print(f"차분 데이터: {len(df_diff)}개")

---

## 퀴즈 1 ⭐ - 정답

**문제**: 상관관계와 인과관계의 차이점을 설명하고, 그랜저 인과성 검정의 귀무가설을 작성하세요.

In [ ]:
# 퀴즈 1 정답

# 상관관계: 두 변수가 함께 "움직이"는 것
# 인과관계: 한 변수가 다른 변수에 "영향"을 주는 것

# 그랜저 인과성 귀무가설 (H0):
# "X가 Y를 그랜저-'원인'하지 않는다"
# = "X의 과거값이 Y 예측에 '도움'이 안 된다"

# 정답 출력
print("=" * 60)
print("상관관계 vs 인과관계")
print("=" * 60)
print()
print("1. 상관관계 (Correlation):")
print("   - 두 변수가 함께 '움직이는' 것")
print("   - 예: 아이스크림 판매량 ↔ 익사 사고 (여름이라는 숨은 변수)")
print("   - X가 증가할 때 Y도 증가/감소하는 '패턴'")
print()
print("2. 인과관계 (Causality):")
print("   - 한 변수가 다른 변수에 '영향'을 주는 것")
print("   - 예: 광고비 → 매출 (광고가 매출을 일으킴)")
print("   - X의 변화가 Y의 변화를 '야기'함")
print()
print("=" * 60)
print("그랜저 인과성 귀무가설 (H0)")
print("=" * 60)
print()
print("귀무가설 H0: 'X가 Y를 그랜저-원인하지 않는다'")
print("           = 'X의 과거값이 Y 예측에 도움이 안 된다'")
print()
print("대립가설 H1: 'X가 Y를 그랜저-원인한다'")
print("           = 'X의 과거값이 Y 예측에 유의미한 정보를 제공한다'")

In [ ]:
# 테스트: 개념 정리 확인
correlation_definition = "두 변수가 함께 움직이는 것"
causality_definition = "한 변수가 다른 변수에 영향을 주는 것"
h0_granger = "X의 과거값이 Y 예측에 도움이 안 된다"

assert "움직" in correlation_definition, "상관관계는 함께 움직이는 것"
assert "영향" in causality_definition, "인과관계는 영향을 주는 것"
assert "예측" in h0_granger and "도움" in h0_granger, "귀무가설 확인"

print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 상관관계는 "패턴"이고 인과관계는 "메커니즘"입니다
- 그랜저 검정은 "예측 인과성"을 검정합니다 (진짜 인과관계는 아님)

**핵심 개념**:
1. 상관관계: 통계적 연관성 (함께 움직임)
2. 인과관계: 원인-결과 관계 (영향을 미침)
3. 그랜저 인과성: 시간적 선행관계 기반 예측력 검정

**주의사항**:
- "correlation does not imply causation" - 상관관계가 인과관계를 의미하지 않음
- 그랜저 인과성도 엄밀한 인과관계는 아님 (숨은 변수 가능)

---

## 퀴즈 2 ⭐ - 정답

**문제**: 그랜저 인과성 검정 결과 p-value가 0.03이 나왔습니다. 이 결과를 어떻게 해석해야 하나요?

In [ ]:
# 퀴즈 2 정답

# p-value = 0.03 해석
p_value = 0.03
alpha = 0.05  # 유의수준

if p_value < alpha:  # p_value가 alpha보다 "작으면"
    result = "귀무가설 기각"  # 귀무가설을 "기각"한다
    interpretation = "X가 Y를 그랜저-원인한다"  # "원인한다"
else:
    result = "귀무가설 채택"  # 귀무가설을 "채택"한다
    interpretation = "인과관계 없음"  # "없음"

print(f"p-value: {p_value}")
print(f"유의수준(alpha): {alpha}")
print(f"비교: {p_value} < {alpha} = {p_value < alpha}")
print()
print(f"결과: {result}")
print(f"해석: {interpretation}")

In [ ]:
# 테스트: 다양한 p-value에 대한 해석 함수
def interpret_granger_pvalue(p_value, alpha=0.05):
    """그랜저 검정 p-value 해석"""
    if p_value < alpha:
        return "귀무가설 기각 - X가 Y를 그랜저-원인함"
    else:
        return "귀무가설 채택 - 인과관계 없음"

# 테스트 케이스
test_cases = [0.01, 0.03, 0.05, 0.10, 0.50]

print("=== p-value별 해석 ===\n")
for p in test_cases:
    print(f"p-value = {p:.2f}: {interpret_granger_pvalue(p)}")

# 정답 검증
assert "기각" in interpret_granger_pvalue(0.03), "p=0.03은 귀무가설 기각"
assert "채택" in interpret_granger_pvalue(0.10), "p=0.10은 귀무가설 채택"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- p-value와 유의수준(alpha) 비교
- p < alpha이면 귀무가설 기각

**핵심 개념**:
| p-value | 비교 | 결과 | 해석 |
|---------|------|------|------|
| 0.03 | 0.03 < 0.05 | 귀무가설 기각 | 인과관계 있음 |
| 0.10 | 0.10 > 0.05 | 귀무가설 채택 | 인과관계 없음 |

**실무 팁**:
- 여러 시차(lag)에서 검정하여 일관된 결과 확인
- 유의수준은 보통 0.05 또는 0.01 사용
- 양방향 검정으로 피드백 관계도 확인

---

## 퀴즈 3 ⭐⭐ - 정답

**문제**: statsmodels의 grangercausalitytests 함수를 사용하여 그랜저 인과성 검정을 수행하세요.

주의: 함수 입력 순서는 [Y, X]입니다 (Y를 예측할 때 X가 도움이 되는지 검정)

In [ ]:
# 퀴즈 3 정답
from statsmodels.tsa.stattools import grangercausalitytests

# "Money가 Spending을 그랜저-원인하는가?" 검정
# => Spending을 예측할 때 Money가 도움이 되는가?

# 핵심: grangercausalitytests 입력 순서는 [결과변수(Y), 원인변수(X)]
# "X가 Y를 원인하는가?" 검정 시 [Y, X] 순서로 입력

test_data = df_diff[['Spending', 'Money']]  # [결과변수, 원인변수] 순서!

# 검정 수행 (maxlag=5)
print("=== Money → Spending 그랜저 인과성 검정 ===")
print("(Spending을 예측할 때 Money가 도움이 되는지 검정)\n")

results = grangercausalitytests(test_data, maxlag=5, verbose=True)

In [ ]:
# 테스트: 결과 정리
print("\n=== p-value 요약 ===")
for lag in range(1, 6):
    p_val = results[lag][0]['ssr_ftest'][1]
    sig = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
    print(f"Lag {lag}: p-value = {p_val:.4f} {sig}")

# 검증: results가 dict이고 lag별 결과가 있어야 함
assert isinstance(results, dict), "결과는 dictionary여야 함"
assert len(results) == 5, "maxlag=5이므로 5개 결과"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `grangercausalitytests(data, maxlag)`의 data 순서가 중요
- "X가 Y를 원인하는가?" 검정 시 `data = [[Y, X]]` 순서

**핵심 개념**:
```python
# Money → Spending 검정 (Money가 Spending을 원인하는가?)
grangercausalitytests(df[['Spending', 'Money']], maxlag=5)
#                       ↑ 결과변수   ↑ 원인변수
```

**자주 하는 실수**:
- [원인, 결과] 순서로 잘못 입력하는 경우
- 정상성 확보 없이 원본 데이터로 검정하는 경우

**실무 팁**:
- 결과에서 `ssr_ftest`의 p-value를 주로 사용
- 여러 시차 중 일관되게 유의한지 확인

---

## 퀴즈 4 ⭐⭐ - 정답

**문제**: VAR 모델과 ARIMA 모델의 차이점을 코드로 비교하세요.

In [ ]:
# 퀴즈 4 정답

# ARIMA: 단변량 (1개 변수)
from statsmodels.tsa.arima.model import ARIMA
# arima_model = ARIMA(df['Money'], order=(1,1,1))  # 단일 열

# VAR: 다변량 (2개 이상 변수)
from statsmodels.tsa.api import VAR
# var_model = VAR(df[['Money', 'Spending']])  # 여러 열

# VAR의 장점은?
# 변수 간 "상호작용"을 반영하여 예측

# 비교 테이블 출력
print("=" * 70)
print("ARIMA vs VAR 비교")
print("=" * 70)
print()
print(f"{'특성':<20} {'ARIMA (단변량)':<25} {'VAR (다변량)'}")
print("-" * 70)
print(f"{'변수 수':<20} {'1개':<25} {'2개 이상'}")
print(f"{'예측 대상':<20} {'Y의 미래값만':<25} {'Y1, Y2, ... 모두'}")
print(f"{'상호작용':<20} {'없음':<25} {'변수 간 영향 반영'}")
print(f"{'활용 예시':<20} {'단일 KPI 예측':<25} {'연관 지표 동시 예측'}")
print(f"{'코드 입력':<20} {'df[col]':<25} {'df[[col1, col2, ...]]}'}")

In [ ]:
# 테스트: 실제 코드 예시
print("=== ARIMA 코드 예시 ===")
print("from statsmodels.tsa.arima.model import ARIMA")
print("model = ARIMA(df['Money'], order=(1,1,1))")
print("fitted = model.fit()")
print("forecast = fitted.forecast(steps=5)  # Money만 예측")
print()

print("=== VAR 코드 예시 ===")
print("from statsmodels.tsa.api import VAR")
print("model = VAR(df[['Money', 'Spending']])")
print("fitted = model.fit(maxlags=5)")
print("forecast = fitted.forecast(y=last_obs, steps=5)  # 둘 다 예측")

# 검증
var_advantage = "변수 간 상호작용을 반영하여 예측"
assert "상호작용" in var_advantage, "VAR의 핵심 장점"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- ARIMA: 단일 시계열의 자기회귀 + 차분 + 이동평균
- VAR: 여러 시계열 간 상호 의존성 모델링

**핵심 개념**:
| 모델 | 수식 (간략화) |
|------|-------------|
| ARIMA | Y(t) = f(Y(t-1), Y(t-2), ..., error) |
| VAR | Y1(t) = f(Y1(t-1), Y2(t-1), ...)  <br> Y2(t) = f(Y1(t-1), Y2(t-1), ...) |

**실무 팁**:
- 독립적인 단일 KPI 예측: ARIMA
- 서로 영향주는 여러 변수 예측: VAR
- VAR도 정상 시계열 필요 (차분 적용)

---

## 퀴즈 5 ⭐⭐ - 정답

**문제**: VAR 모델의 최적 시차(lag)를 선택하는 코드를 작성하세요.

In [ ]:
# 퀴즈 5 정답
from statsmodels.tsa.api import VAR

# VAR 모델 생성
model = VAR(df_diff)

# 최적 시차 선택
lag_selection = model.select_order(maxlags=15)  # select_order 메서드

# AIC 기준 최적 시차
optimal_lag = lag_selection.aic  # .aic 속성
print(f"AIC 기준 최적 시차: {optimal_lag}")

# BIC 기준 최적 시차
optimal_lag_bic = lag_selection.bic  # .bic 속성
print(f"BIC 기준 최적 시차: {optimal_lag_bic}")

In [ ]:
# 테스트: 전체 시차 선택 결과 확인
print("=== 시차 선택 상세 결과 ===")
print(lag_selection.summary())

# 검증
assert hasattr(lag_selection, 'aic'), "AIC 속성 필요"
assert hasattr(lag_selection, 'bic'), "BIC 속성 필요"
assert hasattr(lag_selection, 'hqic'), "HQIC 속성도 있음"
assert hasattr(lag_selection, 'fpe'), "FPE 속성도 있음"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `VAR(data)` 모델 객체 생성
2. `.select_order(maxlags=N)` 메서드로 시차 선택
3. `.aic`, `.bic` 등 속성으로 최적 시차 확인

**핵심 개념**:
| 기준 | 특징 | 속성 |
|------|------|------|
| AIC | 과적합 허용적 | `.aic` |
| BIC | 단순 모델 선호 | `.bic` |
| HQIC | 중간 | `.hqic` |
| FPE | 예측오차 최소화 | `.fpe` |

**실무 팁**:
- AIC와 BIC가 다르면 해석 가능한 시차 선택
- 시차가 너무 크면 과적합, 너무 작으면 정보 손실

---

## 퀴즈 6 ⭐⭐⭐ - 정답

**문제**: VAR 모델을 학습하고 예측을 수행하는 전체 코드를 작성하세요.

In [ ]:
# 퀴즈 6 정답
from statsmodels.tsa.api import VAR

# 1. 데이터 분할
n_test = 12
train = df_diff[:-n_test]  # 훈련 데이터
test = df_diff[-n_test:]   # 테스트 데이터

print(f"훈련 데이터: {len(train)}개")
print(f"테스트 데이터: {len(test)}개")

# 2. VAR 모델 학습
model = VAR(train)  # 훈련 데이터로 모델 생성

# 최적 시차 선택
optimal_lag = model.select_order(maxlags=15).aic
print(f"최적 시차: {optimal_lag}")

# 모델 학습
fitted = model.fit(optimal_lag)  # 최적 시차로 fit

# 3. 예측 (forecast 메서드 사용)
lag_order = fitted.k_ar  # 학습된 시차 수
forecast_input = train.values[-lag_order:]  # 마지막 lag_order개 데이터
forecast = fitted.forecast(y=forecast_input, steps=n_test)  # n_test 기간 예측

print(f"\n예측 shape: {forecast.shape}")
print(f"예측 결과 (첫 5개):")
print(forecast[:5])

In [ ]:
# 테스트: 예측 결과 검증
assert forecast.shape[0] == n_test, f"예측 개수가 {n_test}개여야 함"
assert forecast.shape[1] == len(df_diff.columns), "변수 개수가 일치해야 함"

# DataFrame으로 변환
df_forecast = pd.DataFrame(
    forecast,
    index=test.index,
    columns=[f"{col}_pred" for col in df_diff.columns]
)
print("=== 예측 결과 DataFrame ===")
print(df_forecast.head())
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 훈련/테스트 데이터 분할
2. `VAR(train)` → `.fit(lag)` 순서로 학습
3. `forecast(y=last_obs, steps=n)` 메서드로 예측

**핵심 개념**:
```python
# VAR 예측에 필요한 것
lag_order = fitted.k_ar           # 학습된 시차 수
forecast_input = train.values[-lag_order:]  # 마지막 lag개 관측값
forecast = fitted.forecast(y=forecast_input, steps=n)
```

**자주 하는 실수**:
- `forecast()`에 y 파라미터 누락
- 잘못된 lag_order 사용
- 테스트 데이터 포함하여 학습

**실무 팁**:
- 차분 데이터로 학습/예측 후 역변환 필요
- steps는 테스트 기간에 맞춤

---

## 퀴즈 7 ⭐⭐⭐ - 정답

**문제**: yfinance를 사용하여 2개 종목의 주가를 수집하고, 양방향 그랜저 인과성 검정을 수행하세요.

In [ ]:
# 퀴즈 7 정답
import yfinance as yf

# 1. 데이터 수집 (한국 주식)
print("주식 데이터 수집 중...")
naver = yf.download('035420.KS', period='2y')['Close']
kakao = yf.download('035720.KS', period='2y')['Close']

# 2. DataFrame 결합
df_stock = pd.DataFrame({
    'NAVER': naver,
    'KAKAO': kakao
}).dropna()

print(f"\n데이터 기간: {df_stock.index[0].date()} ~ {df_stock.index[-1].date()}")
print(f"데이터 개수: {len(df_stock)}개")

# 수익률로 변환 (정상성 확보)
df_returns = df_stock.pct_change().dropna() * 100  # 백분율

# 정상성 확인
print("\n=== 정상성 검정 ===")
for col in df_returns.columns:
    adf_result = adfuller(df_returns[col])
    print(f"{col}: ADF={adf_result[0]:.4f}, p-value={adf_result[1]:.4f} -> {'정상' if adf_result[1] < 0.05 else '비정상'}")

In [ ]:
# 3. 양방향 그랜저 검정
def simple_granger_test(data, cause, effect, maxlag=5):
    """간단한 그랜저 검정 결과 요약"""
    print(f"\n=== {cause} → {effect} 그랜저 검정 ===")
    test_data = data[[effect, cause]].dropna()  # [결과, 원인] 순서
    results = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
    
    min_p = 1.0
    best_lag = 1
    
    for lag in range(1, maxlag + 1):
        p_val = results[lag][0]['ssr_ftest'][1]
        sig = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
        print(f"  Lag {lag}: p = {p_val:.4f} {sig}")
        if p_val < min_p:
            min_p = p_val
            best_lag = lag
    
    if min_p < 0.05:
        print(f"  => 결론: {cause}가 {effect}를 그랜저-원인함 (lag={best_lag}, p={min_p:.4f})")
    else:
        print(f"  => 결론: 인과관계 없음 (p >= 0.05)")
    
    return min_p

# 양방향 검정 수행
p1 = simple_granger_test(df_returns, 'NAVER', 'KAKAO')  # NAVER → KAKAO
p2 = simple_granger_test(df_returns, 'KAKAO', 'NAVER')  # KAKAO → NAVER

In [ ]:
# 테스트: 검정 완료 확인
assert 'NAVER' in df_returns.columns, "NAVER 데이터 필요"
assert 'KAKAO' in df_returns.columns, "KAKAO 데이터 필요"
assert len(df_returns) > 100, "충분한 데이터 필요"

print("\n" + "="*60)
print("양방향 그랜저 인과성 검정 완료!")
print("="*60)
print(f"NAVER → KAKAO: {'인과관계 있음' if p1 < 0.05 else '인과관계 없음'}")
print(f"KAKAO → NAVER: {'인과관계 있음' if p2 < 0.05 else '인과관계 없음'}")

print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. yfinance로 주가 수집 (한국 종목은 `.KS` 접미사)
2. 수익률로 변환 (`.pct_change()`) - 정상성 확보
3. 양방향 검정: A→B, B→A 모두 확인

**핵심 개념**:
- 수익률 변환 이유: 주가는 비정상, 수익률은 정상
- 양방향 검정으로 상호 영향 파악

**실무 팁**:
- 동일 업종 종목 간 리드-래그 관계 분석
- 효율적 시장에서는 인과관계 약할 수 있음
- 그랜저 인과성 ≠ 투자 수익 보장

---

## 퀴즈 8 ⭐⭐⭐⭐ - 정답

**문제**: 차분된 VAR 예측 결과를 원본 스케일로 역변환하는 함수를 구현하세요.

In [ ]:
# 퀴즈 8 정답

def invert_difference(original_series, forecast_diff):
    """
    차분된 예측값을 원본 스케일로 역변환
    
    Parameters:
    -----------
    original_series : pd.Series
        원본 시계열 (차분 전)
    forecast_diff : np.array
        차분된 예측값
    
    Returns:
    --------
    np.array : 원본 스케일 예측값
    """
    # 마지막 원본값
    last_value = original_series.iloc[-1]  # iloc[-1]로 마지막 값 가져오기
    
    # 누적합으로 역변환
    # diff(t) = Y(t) - Y(t-1) 이므로
    # Y(t) = Y(t-1) + diff(t)
    # => Y(t) = last_value + cumsum(diff[1:t])
    forecast_original = np.cumsum(forecast_diff) + last_value  # cumsum() 사용
    
    return forecast_original

# 함수 정의 확인
print("invert_difference 함수 정의 완료!")

In [ ]:
# 테스트 1: 간단한 예제
print("=== 테스트 1: 간단한 예제 ===")
original = pd.Series([100, 110, 125, 140])  # 원본 시계열
diff_forecast = np.array([15, 12, 18])  # 예측된 차분

result = invert_difference(original, diff_forecast)

print(f"원본 시계열: {original.values}")
print(f"마지막 값: {original.iloc[-1]}")
print(f"예측된 차분: {diff_forecast}")
print(f"역변환 결과: {result}")

# 검증
# 140 + 15 = 155, 155 + 12 = 167, 167 + 18 = 185
expected = np.array([155, 167, 185])
assert np.allclose(result, expected), f"예상: {expected}, 실제: {result}"
print(f"예상값: {expected}")
print("테스트 1 통과!")

In [ ]:
# 테스트 2: 실제 VAR 예측에 적용
print("\n=== 테스트 2: VAR 예측 역변환 ===")

# VAR 예측 (차분 데이터)
n_test = 12
train_diff = df_diff[:-n_test]
train_orig = df_econ[:-n_test]

model = VAR(train_diff)
fitted = model.fit(model.select_order(maxlags=10).aic)

lag_order = fitted.k_ar
forecast_diff = fitted.forecast(y=train_diff.values[-lag_order:], steps=n_test)

# 각 변수에 역변환 적용
forecast_original = {}
for i, col in enumerate(train_diff.columns):
    forecast_original[col] = invert_difference(
        train_orig[col], 
        forecast_diff[:, i]
    )
    print(f"{col} 역변환 완료")

print("\n테스트 2 통과!")

### 💡 풀이 설명

**접근 방법**:
- 1차 차분: `diff(t) = Y(t) - Y(t-1)`
- 역변환: `Y(t) = Y(t-1) + diff(t)`
- 연속 역변환: `cumsum(diff) + last_value`

**핵심 개념**:
```
원본:    100, 110, 125, 140 (마지막=140)
차분예측: 15, 12, 18

역변환:
  t1: 140 + 15 = 155
  t2: 155 + 12 = 167  (= 140 + 15 + 12)
  t3: 167 + 18 = 185  (= 140 + 15 + 12 + 18)

=> cumsum([15, 12, 18]) + 140 = [15, 27, 45] + 140 = [155, 167, 185]
```

**자주 하는 실수**:
- `iloc[-1]` 대신 `[-1]` 사용 (인덱스 문제)
- 다중 차분 시 순서 혼동

**실무 팁**:
- 2차 차분이면 역변환도 2번
- DataFrame 전체에 적용하려면 열별로 처리

---

## 퀴즈 9 ⭐⭐⭐⭐ - 정답

**문제**: 여러 변수에 대한 그랜저 인과성 매트릭스를 생성하는 함수를 구현하세요.

In [ ]:
# 퀴즈 9 정답

def granger_matrix(data, variables, maxlag=5, significance=0.05):
    """
    여러 변수 간 그랜저 인과성 매트릭스 생성
    
    Parameters:
    -----------
    data : pd.DataFrame
        시계열 데이터 (정상성 확보 필요)
    variables : list
        분석할 변수 목록
    maxlag : int
        검정할 최대 시차
    significance : float
        유의수준
    
    Returns:
    --------
    pd.DataFrame : 인과성 매트릭스 (행: 원인, 열: 결과)
                   값은 최소 p-value
    """
    n = len(variables)
    matrix = np.ones((n, n))  # 1로 초기화 (대각선은 NaN 처리할 예정)
    
    for i, cause in enumerate(variables):
        for j, effect in enumerate(variables):
            if i == j:
                # 자기 자신: NaN
                matrix[i, j] = np.nan
            else:
                try:
                    # 그랜저 검정 수행
                    # 입력 순서: [결과변수, 원인변수]
                    test_data = data[[effect, cause]].dropna()
                    results = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
                    
                    # 모든 시차에서 최소 p-value 찾기
                    min_p = min([results[lag][0]['ssr_ftest'][1] for lag in range(1, maxlag+1)])
                    matrix[i, j] = min_p
                except:
                    matrix[i, j] = np.nan
    
    # DataFrame으로 변환 (행: 원인, 열: 결과)
    result_df = pd.DataFrame(
        matrix, 
        index=[f"{v} →" for v in variables],  # 원인
        columns=variables  # 결과
    )
    
    return result_df

print("granger_matrix 함수 정의 완료!")

In [ ]:
# 테스트 1: 2변수 데이터
print("=== 테스트 1: 경제 데이터 (2변수) ===")
matrix_econ = granger_matrix(df_diff, ['Money', 'Spending'], maxlag=5)
print(matrix_econ.round(4))
print("\n해석: 행(원인) → 열(결과)의 p-value")
print("p < 0.05이면 인과관계 있음")

In [ ]:
# 테스트 2: 3변수 이상 데이터 (주식 수익률)
import yfinance as yf

print("\n=== 테스트 2: 주식 수익률 (3변수) ===")

# 데이터 수집
symbols = {'SAMSUNG': '005930.KS', 'SKHYNIX': '000660.KS', 'LG': '066570.KS'}
stock_data = {}

print("데이터 수집 중...")
for name, ticker in symbols.items():
    stock_data[name] = yf.download(ticker, period='2y', progress=False)['Close']

df_stocks = pd.DataFrame(stock_data).dropna()
df_stock_returns = df_stocks.pct_change().dropna() * 100

# 그랜저 매트릭스 생성
matrix_stocks = granger_matrix(df_stock_returns, list(symbols.keys()), maxlag=5)
print("\n그랜저 인과성 매트릭스 (p-values):")
print(matrix_stocks.round(4))

In [ ]:
# 시각화: Heatmap
def plot_granger_heatmap(matrix, title="그랜저 인과성 매트릭스"):
    """그랜저 매트릭스 히트맵 시각화"""
    # NaN을 1로 대체 (대각선)
    matrix_plot = matrix.fillna(1)
    
    fig = go.Figure(data=go.Heatmap(
        z=matrix_plot.values,
        x=matrix_plot.columns,
        y=matrix_plot.index,
        colorscale='RdYlGn_r',  # 빨간색이 낮은 p-value (유의)
        zmin=0, zmax=0.1,  # 0~0.1 범위
        text=matrix_plot.round(3).values,
        texttemplate='%{text}',
        textfont={'size': 12},
        hoverongaps=False,
        colorbar=dict(title='p-value')
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title='결과 변수 (Effect)',
        yaxis_title='원인 변수 (Cause)',
        height=400
    )
    
    return fig

fig = plot_granger_heatmap(matrix_stocks, "주식 그랜저 인과성 (p < 0.05이면 유의)")
fig.show()

print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. N x N 매트릭스 생성 (N = 변수 개수)
2. 모든 (원인, 결과) 쌍에 대해 그랜저 검정
3. 최소 p-value를 매트릭스에 저장
4. 대각선(자기 자신)은 NaN 처리

**핵심 개념**:
```
매트릭스 해석:
       Money  Spending
Money →   NaN    0.02   <- Money가 Spending을 원인함 (p=0.02)
Spending→ 0.15    NaN   <- Spending은 Money를 원인하지 않음
```

**실무 팁**:
- 히트맵으로 시각화하면 패턴 파악 용이
- p < 0.05인 셀만 인과관계 존재
- 양방향 모두 유의하면 피드백 관계

---

## 퀴즈 10 ⭐⭐⭐⭐⭐ - 정답

**문제**: 다변량 시계열 분석 파이프라인 클래스를 구현하세요.

In [ ]:
# 퀴즈 10 정답

class MultivariateAnalyzer:
    """
    다변량 시계열 분석 파이프라인
    
    VAR 모델 학습, 그랜저 인과성 검정, 예측 및 시각화를 제공합니다.
    """
    
    def __init__(self, maxlag=15):
        """
        Parameters:
        -----------
        maxlag : int
            최적 시차 탐색 시 최대 시차
        """
        self.maxlag = maxlag
        self.model = None
        self.fitted = None
        self.data_original = None
        self.data_diff = None
        self.train_original = None
        self.train_diff = None
        self.test_original = None
        self.test_diff = None
        self.forecast_diff = None
        self.forecast_original = None
        self.optimal_lag = None
        self.n_test = None
    
    def fit(self, data, test_size=12):
        """
        데이터 분할, 정상성 변환, VAR 학습
        
        Parameters:
        -----------
        data : pd.DataFrame
            원본 시계열 데이터 (여러 열)
        test_size : int
            테스트 데이터 크기
        """
        self.n_test = test_size
        self.data_original = data.copy()
        
        # 1. 정상성 변환 (1차 차분)
        self.data_diff = data.diff().dropna()
        
        # 2. 데이터 분할
        self.train_original = data.iloc[:-test_size]
        self.test_original = data.iloc[-test_size:]
        self.train_diff = self.data_diff.iloc[:-test_size]
        self.test_diff = self.data_diff.iloc[-test_size:]
        
        # 3. VAR 모델 학습
        self.model = VAR(self.train_diff)
        self.optimal_lag = self.model.select_order(maxlags=self.maxlag).aic
        self.fitted = self.model.fit(self.optimal_lag)
        
        print(f"=== VAR 모델 학습 완료 ===")
        print(f"변수: {list(data.columns)}")
        print(f"훈련 데이터: {len(self.train_diff)}개")
        print(f"테스트 데이터: {len(self.test_diff)}개")
        print(f"최적 시차 (AIC): {self.optimal_lag}")
        
        return self
    
    def granger_test(self, cause, effect, maxlag=None):
        """
        그랜저 인과성 검정
        
        Parameters:
        -----------
        cause : str
            원인 변수
        effect : str
            결과 변수
        maxlag : int
            최대 시차 (None이면 self.maxlag 사용)
        """
        if maxlag is None:
            maxlag = min(self.maxlag, 10)
        
        print(f"\n=== 그랜저 인과성 검정: {cause} → {effect} ===")
        
        # [결과, 원인] 순서로 입력
        test_data = self.data_diff[[effect, cause]].dropna()
        results = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
        
        # 결과 정리
        print(f"{'Lag':<5} {'F-stat':<12} {'p-value':<12} {'유의성'}")
        print("-" * 40)
        
        min_p = 1.0
        best_lag = 1
        
        for lag in range(1, maxlag + 1):
            f_stat = results[lag][0]['ssr_ftest'][0]
            p_val = results[lag][0]['ssr_ftest'][1]
            sig = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
            print(f"{lag:<5} {f_stat:<12.4f} {p_val:<12.4f} {sig}")
            
            if p_val < min_p:
                min_p = p_val
                best_lag = lag
        
        print(f"\n결론: ", end="")
        if min_p < 0.05:
            print(f"{cause}가 {effect}를 그랜저-원인함 (lag={best_lag}, p={min_p:.4f})")
        else:
            print(f"인과관계 없음 (p >= 0.05)")
        
        return {'min_p': min_p, 'best_lag': best_lag, 'results': results}
    
    def predict(self, steps=None):
        """
        VAR 예측 (원본 스케일 반환)
        
        Parameters:
        -----------
        steps : int
            예측 기간 (None이면 test_size 사용)
        """
        if steps is None:
            steps = self.n_test
        
        # 차분 예측
        lag_order = self.fitted.k_ar
        forecast_input = self.train_diff.values[-lag_order:]
        self.forecast_diff = self.fitted.forecast(y=forecast_input, steps=steps)
        
        # 원본 스케일로 역변환
        last_values = self.train_original.iloc[-1].values
        self.forecast_original = np.cumsum(self.forecast_diff, axis=0) + last_values
        
        # DataFrame으로 변환
        forecast_index = self.test_original.index[:steps] if steps <= len(self.test_original) \
                        else pd.date_range(self.test_original.index[0], periods=steps, freq='MS')
        
        forecast_df = pd.DataFrame(
            self.forecast_original,
            index=forecast_index,
            columns=[f"{col}_forecast" for col in self.train_original.columns]
        )
        
        print(f"\n=== {steps}기간 예측 완료 ===")
        print(forecast_df.head())
        
        return forecast_df
    
    def evaluate(self):
        """
        예측 성능 평가 (MAE, RMSE, MAPE)
        """
        if self.forecast_original is None:
            print("먼저 predict()를 실행하세요.")
            return None
        
        metrics = {}
        print("\n=== 예측 성능 평가 ===")
        
        for i, col in enumerate(self.train_original.columns):
            actual = self.test_original[col].values[:len(self.forecast_original)]
            predicted = self.forecast_original[:, i]
            
            mae = np.mean(np.abs(actual - predicted))
            rmse = np.sqrt(np.mean((actual - predicted) ** 2))
            mape = np.mean(np.abs((actual - predicted) / actual)) * 100
            
            metrics[col] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}
            
            print(f"\n{col}:")
            print(f"  MAE:  {mae:.2f}")
            print(f"  RMSE: {rmse:.2f}")
            print(f"  MAPE: {mape:.2f}%")
        
        return metrics
    
    def plot(self):
        """
        예측 결과 시각화
        """
        if self.forecast_original is None:
            print("먼저 predict()를 실행하세요.")
            return None
        
        n_vars = len(self.train_original.columns)
        fig = make_subplots(rows=n_vars, cols=1, shared_xaxes=True,
                           subplot_titles=list(self.train_original.columns))
        
        colors = ['blue', 'green', 'red', 'purple', 'orange']
        
        for i, col in enumerate(self.train_original.columns):
            color = colors[i % len(colors)]
            
            # 실제 값 (전체)
            fig.add_trace(go.Scatter(
                x=self.data_original.index,
                y=self.data_original[col],
                mode='lines', name=f'{col} 실제',
                line=dict(color=color)
            ), row=i+1, col=1)
            
            # 예측 값
            forecast_index = self.test_original.index[:len(self.forecast_original)]
            fig.add_trace(go.Scatter(
                x=forecast_index,
                y=self.forecast_original[:, i],
                mode='lines', name=f'{col} 예측',
                line=dict(color=color, dash='dash')
            ), row=i+1, col=1)
        
        fig.update_layout(
            title='VAR 모델 예측 결과',
            height=300 * n_vars,
            showlegend=True
        )
        
        return fig

print("MultivariateAnalyzer 클래스 정의 완료!")

In [ ]:
# 테스트: 전체 파이프라인 실행
print("=" * 70)
print("MultivariateAnalyzer 전체 테스트")
print("=" * 70)

# 1. 인스턴스 생성
analyzer = MultivariateAnalyzer(maxlag=15)

# 2. 모델 학습
analyzer.fit(df_econ, test_size=12)

In [ ]:
# 3. 그랜저 인과성 검정
granger_result = analyzer.granger_test('Money', 'Spending')

In [ ]:
# 4. 예측
forecast_df = analyzer.predict()

In [ ]:
# 5. 성능 평가
metrics = analyzer.evaluate()

In [ ]:
# 6. 시각화
fig = analyzer.plot()
fig.show()

In [ ]:
# 테스트: 클래스 속성 검증
assert analyzer.fitted is not None, "모델 학습 필요"
assert analyzer.forecast_original is not None, "예측 필요"
assert isinstance(metrics, dict), "메트릭은 dict"

# 메서드 존재 확인
assert hasattr(analyzer, 'fit'), "fit 메서드 필요"
assert hasattr(analyzer, 'granger_test'), "granger_test 메서드 필요"
assert hasattr(analyzer, 'predict'), "predict 메서드 필요"
assert hasattr(analyzer, 'evaluate'), "evaluate 메서드 필요"
assert hasattr(analyzer, 'plot'), "plot 메서드 필요"

print("\n" + "=" * 70)
print("모든 테스트 통과!")
print("=" * 70)

### 💡 풀이 설명

**접근 방법**:
1. 클래스로 상태(data, model, forecast) 관리
2. 파이프라인: fit → granger_test → predict → evaluate → plot
3. 자동 차분 및 역변환 처리

**핵심 메서드**:
| 메서드 | 기능 | 입력 | 출력 |
|--------|------|------|------|
| `fit()` | 학습 | 원본 데이터 | self |
| `granger_test()` | 인과성 검정 | 원인, 결과 | p-value |
| `predict()` | 예측 | steps | 원본 스케일 예측 |
| `evaluate()` | 성능 평가 | - | MAE, RMSE, MAPE |
| `plot()` | 시각화 | - | Plotly Figure |

**확장 아이디어**:
- `granger_matrix()` 메서드 추가
- 충격반응분석(IRF) 추가
- 예측구간 추가
- 롤링 예측 기능

---

## 📊 학습 정리

---

### 퀴즈 난이도별 핵심 정리

| 난이도 | 퀴즈 | 핵심 개념 |
|--------|------|----------|
| ⭐ | 1-2 | 상관/인과 구분, p-value 해석 |
| ⭐⭐ | 3-5 | grangercausalitytests, VAR vs ARIMA, select_order |
| ⭐⭐⭐ | 6-7 | VAR 학습/예측, 주식 그랜저 분석 |
| ⭐⭐⭐⭐ | 8-9 | 차분 역변환, 그랜저 매트릭스 |
| ⭐⭐⭐⭐⭐ | 10 | MultivariateAnalyzer 클래스 |

### 실무 활용 체크리스트

- [ ] 정상성 확보 후 검정/모델링 (차분 필수)
- [ ] 그랜저 검정 시 [결과, 원인] 순서 주의
- [ ] 양방향 검정으로 피드백 관계 확인
- [ ] VAR 예측 후 반드시 역변환
- [ ] 그랜저 인과성 ≠ 진짜 인과관계 명심